In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"D:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 1000)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 87.70it/s]


Total cost = 1427.6237184851632

Cluster 142 assigned to Class 1. Cost: 3.212969949002212
Cluster 279 assigned to Class 2. Cost: 2.5483910346318357
Cluster 163 assigned to Class 3. Cost: 2.4162964800798137
Cluster 85 assigned to Class 4. Cost: 2.149926874179229
Cluster 167 assigned to Class 5. Cost: 2.511771015059462
Cluster 191 assigned to Class 6. Cost: 1.5560911907190502
Cluster 271 assigned to Class 11. Cost: 1.7096202277172383
Cluster 117 assigned to Class 14. Cost: 1.5559653197707144
Cluster 76 assigned to Class 18. Cost: 3.350297290368529
Cluster 115 assigned to Class 20. Cost: 2.9657373155437448
Cluster 222 assigned to Class 21. Cost: 2.3643463224559502
Cluster 83 assigned to Class 22. Cost: 2.3709165636615066
Cluster 195 assigned to Class 23. Cost: 3.104733465349717
Cluster 177 assigned to Class 24. Cost: 2.341410164408899
Cluster 181 assigned to Class 27. Cost: 2.260532474905334
Cluster 350 assigned to Class 29. Cost: 3.888970665562307
Cluster 235 assigned to Class 31. Cost: 

Cluster 218 assigned to Class 789. Cost: 1.5490162996005383
Cluster 288 assigned to Class 791. Cost: 3.747831560044584
Cluster 263 assigned to Class 794. Cost: 1.7495944394004506
Cluster 136 assigned to Class 796. Cost: 6.4459268614478535
Cluster 216 assigned to Class 798. Cost: 3.407905953180907
Cluster 36 assigned to Class 799. Cost: 3.1172142811246992
Cluster 392 assigned to Class 800. Cost: 3.3591731874174844
Cluster 27 assigned to Class 802. Cost: 6.147427018763356
Cluster 249 assigned to Class 803. Cost: 4.453909245376958
Cluster 360 assigned to Class 808. Cost: 4.0895484428983515
Cluster 112 assigned to Class 809. Cost: 2.586810610313009
Cluster 328 assigned to Class 812. Cost: 2.603314932289297
Cluster 121 assigned to Class 814. Cost: 3.9878154979136102
Cluster 234 assigned to Class 815. Cost: 3.1995070070113942
Cluster 178 assigned to Class 817. Cost: 3.8861380329005977
Cluster 363 assigned to Class 818. Cost: 5.054082408627247
Cluster 77 assigned to Class 820. Cost: 1.9744811

In [6]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
